# NumPy Monte Carlo Simulation

In [1]:
import math
import random
import time

## Standard Implementations

In [2]:
def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When box_length is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : list
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length

    Returns
    -------
    distance : float
        The distance between the two points accounting for periodic boundaries
    """
    distance = 0
        
    for i in range(3):
        hold_dist = abs(coord2[i] - coord1[i])
    
        if (box_length):    
            if hold_dist > box_length/2:
                hold_dist = hold_dist - (box_length * round(hold_dist/box_length))
        distance += math.pow(hold_dist, 2)

    return math.sqrt(distance)


In [3]:
pt1= [0,0,0]
pt2= [8,0,0]


calculate_distance(pt1,pt2,3.3)

1.4000000000000004

In [4]:
def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy

In [5]:
assert calculate_LJ(1) == 0
assert calculate_LJ(math.pow(2, (1/6))) == -1.0

calculate_LJ(4.0)
#assert calculate_LJ(3.0) == -0.1910413370565145

-0.0009763240814208984

In [6]:
def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : list
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length)

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
    return total_energy




In [7]:
coordinates = [[0, 0, 0], [0, math.pow(2, 1/6), 0], [0, 2*math.pow(2, 1/6), 0]]

test_energy =  calculate_total_energy(coordinates, 10, 3.0) 

print(test_energy)

assert test_energy == -2.031005859375

-2.031005859375


In [8]:
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system)
    
    Parameters
    ----------
    coordinates : list
        The coordinates for all the particles in the system.
        
    i_particle : int
        The particle number for which to calculate the energy.
        
    cutoff : float
        The simulation cutoff. Beyond this distance, interactions are not calculated.
    
    box_length : float
        The length of the box for periodic bounds
        
    Returns
    -------
    e_total : float
        The pairwise interaction energy of the ith particles with all other particles in the system
    """
    
    e_total = 0.0
    #creates a list of the coordinates for the i_particle
    i_position = coordinates[i_particle]
    
    num_atoms = len(coordinates)
    
    for j_particle in range(num_atoms):
        
        if i_particle != j_particle:
            #creates a list of coordinates for the j_particle
            j_position = coordinates[j_particle]
            rij = calculate_distance(i_position, j_position, box_length)
            
            if rij < cutoff:
                e_pair = calculate_LJ(rij)
                e_total += e_pair
    
    return e_total
    

In [9]:
coordinates = [[0, 0, 0], [0, 0, 2**(1/6)], [0, 0, 2*(2**(1/6))]]

assert calculate_pair_energy(coordinates, 1, 10, 3) == -2

assert calculate_pair_energy(coordinates, 0, 10, 3) == calculate_pair_energy(coordinates, 2, 10, 3)

assert calculate_pair_energy(coordinates, 0, 10, 2) == -1

In [10]:
def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    return atomic_coordinates, box_length



In [11]:
def calculate_tail_correction(box_legth, n_particles, cutoff):
    """
    Calculates the tail correction
    
    Parameters
    ----------
    box_legth : float
        The distance between the particles in reduced units.
    n_particles : int
        The number of particles.
    cutoff : float
        The cutoff value.
    """
    pi_term = (8 * math.pi * math.pow(n_particles, 2)) / (3 * math.pow(box_legth, 3))
    r9_term = (1/3) * (math.pow(1/cutoff, 9))
    r3_term = math.pow(1/cutoff, 3)
    
    tail_correction = pi_term * (r9_term - r3_term)
    
    return tail_correction

In [12]:
def accept_or_reject(delta_U, beta):
    """
    Accept or reject a move based on the Metropolis criterion.
    
    Parameters
    ----------
    detlta_U : float
        The change in energy for moving system from state m to n.
    beta : float
        1/temperature
    
    Returns
    -------
    boolean
        Whether the move is accepted.
    """
    if delta_U <= 0.0:
        accept = True
    else:
        #Generate a random number on (0,1)
        random_number = random.random()
        p_acc = math.exp(-beta*delta_U)
        
        if random_number < p_acc:
            accept = True
        else:
            accept = False
    return accept

In [13]:
# Set globals the same to compare time

# Read or generate initial coordinates
coordinates, box_length = read_xyz('../../lj_sample_configurations/lj_sample_config_periodic1.txt')
print(box_length)
print(len(coordinates))
# Set simulation parameters
reduced_temperature = 0.9
num_steps = 5000
max_displacement = 0.1
cutoff = 3
    
#how often to print an update
freq = 1000

# Calculated quantities
beta = 1 / reduced_temperature
num_particles = len(coordinates)

10.0
800


In [32]:
start_std = time.time()

#Energy calculations
total_energy = calculate_total_energy(coordinates, box_length, cutoff)
print(total_energy)
total_correction = calculate_tail_correction(num_particles, box_length, cutoff)
print(total_correction)
total_energy += total_correction


for step in range(num_steps):
    # 1. Randomly pick one of the particles.
    random_particle = random.randrange(num_particles)
    
    # 2. Calculate the interaction energy of the selected particle with the system.
    current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
    
    # 3. Generate a random x, y, z displacement.
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)
    
    # 4. Modify the coordinate of Nth particle by generated displacements.
    coordinates[random_particle][0] += x_rand
    coordinates[random_particle][1] += y_rand
    coordinates[random_particle][2] += z_rand
    
    # 5. Calculate the interaction energy of the moved particle with the system and store this value.
    proposed_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
    delta_energy = proposed_energy - current_energy
    
    # 6. Calculate if we accept the move based on energy difference.
    accept = accept_or_reject(delta_energy, beta)
    
    # 7. If accepted, move the particle.
    if accept:
        total_energy += delta_energy
    else:
        #Move not accepted, roll back coordinates
        coordinates[random_particle][0] -= x_rand
        coordinates[random_particle][1] -= y_rand
        coordinates[random_particle][2] -= z_rand
    
    # 8. Print the energy if step is a multiple of freq.
    if step % freq == 0:
        print(step, total_energy/num_particles)
        
end_std = time.time()
elapsed_std = end_std - start_std
print(elapsed_std)

-4351.540194543858
-6.057400016606341e-08
0 -5.4389152161252365
1000 -5.432619114132905
2000 -5.431427240538102
3000 -5.423851097764867
4000 -5.433537881693178
84.82170486450195


# NumPy Implementations

In [15]:
import numpy as np
pt1 = np.array([0,0,0])
pt2 = np.array([0,8,0])


calculate_distance(pt1,pt2)

8.0

In [16]:
dimensional_distance = pt2-pt1
print (dimensional_distance)

[0 8 0]


In [17]:
dd2 = dimensional_distance **2
print(dd2)

dd2_sum=dd2.sum()
distance = math.sqrt(dd2_sum)
print (distance)

[ 0 64  0]
8.0


In [18]:
def calculate_distance_np(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When box_length is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : np.array
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length

    Returns
    -------
    distance : float
        The distance between the two points accounting for periodic boundaries
    """
    coord_dist=coord1-coord2
    if box_length:    
            coord_dist = coord_dist - (box_length * np.round(coord_dist/box_length))
    if coord_dist.ndim <2:
        #reshaping the array, make it have 2 dimensions
        coord_dist = coord_dist.reshape (1,-1)
    coord_dist=coord_dist**2
    dist_sum= coord_dist.sum(axis=1)
    distance = np.sqrt(dist_sum)
    return distance


In [19]:
calculate_distance_np(pt1,pt2)

array([8.])

In [20]:

dist1 = calculate_distance_np(pt1,pt2,2.5)
dist2 = calculate_distance_np(pt1,pt2,2.5)

assert dist1 == dist2


In [21]:
pts1= [[0,0,0],[0,0,1],[0,0,2]]
pts2= [[8,0,0],[8,0,1],[8,0,2]]

pt1np = np.array(pts1)
pt2np = np.array(pts2)

calculate_distance_np(pt1np,pt2np,3.3)

array([1.4, 1.4, 1.4])

In [22]:
def calculate_lj_np(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : np.array of r_ij
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    r1_term = 1 / r_ij
    r6_term = np.power(r1_term, 6)
    r12_term = np.power(r6_term, 2)
    
    pairwise_energy_np = 4 * (r12_term - r6_term)
    
    pairwise_energy = pairwise_energy_np.sum()
    return pairwise_energy

In [23]:
assert calculate_lj_np(1) == 0
assert calculate_lj_np(math.pow(2, (1/6))) == -1

In [24]:
distances = [[1,1,1],[2,3,2],[4,8,2]]

distances_np = np.array(distances)

assert calculate_lj_np(distances_np) == -0.1910413370565145

In [25]:
def calculate_total_energy_np(coordinates_np, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates_np : np.array
        A nested list containing the x, y,z coordinate for each particle
    
    box_length : float
        The length of the box. Assumes cubic box.
    
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    total_energy = 0
    energy = 0
    
    for i in coordinates_np:
        c_np = np.array(i)
        
        # Get distance
        distance_np = calculate_distance_np(coordinates_np, c_np, box_length)
        
        # Cutoff
        distance = distance_np[distance_np < cutoff]
        
        # Non zero
        distance_nz = distance[distance != 0]
        
        total_energy += calculate_lj_np(distance_nz)

    return total_energy.sum()/2

In [26]:
def calculate_total_energy_npnl(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates_np : np.array
        A nested list containing the x, y,z coordinate for each particle
    
    box_length : float
        The length of the box. Assumes cubic box.
    
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    total_energy = 0
    distance = 0
    
    if(len(coordinates) % 2 != 0):
        coordinates.append(coordinates[0])
    
    coordinates_np = np.array(coordinates)
    
    coords = np.array_split(coordinates_np, 2)
    
    distance = calculate_distance_np(coords[0], coords[1], box_length)
    
    energy = distance[distance < cutoff]
    e_positive = distance[distance != 0]
        
    lj = calculate_lj_np(e_positive)    
    

    return lj

In [27]:
coordinates = [[0, 0, 0], [0, math.pow(2, 1/6), 0], [0, 2*math.pow(2, 1/6), 0]]


test_energy = calculate_total_energy_np(coordinates, 10, 3.0)

print(test_energy)


-2.031005859375


In [28]:
coordinates = [[0, 0, 0], [0, math.pow(2, 1/6), 0], [0, 2*math.pow(2, 1/6), 0]]


coords_np = np.array(coordinates)
print(coords_np)


test_energy = calculate_total_energy_npnl(coordinates, 10, 3.0)

print(test_energy)

[[0.         0.         0.        ]
 [0.         1.12246205 0.        ]
 [0.         2.2449241  0.        ]]
-1.031005859375


In [33]:
 def calculate_pair_energy_np(coordinates_np, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system)
    
    Parameters
    ----------
    coordinates_np : np.array
        The coordinates for all the particles in the system.
    
    i_particle : int
        The particle number for which to calculate the energy.
        
    cutoff : float
        The simulation cutoff. Beyond this distance, interactions are not calculated.
    
    box_length : float
        The length of the box for periodic bounds
        
    Returns
    -------
    e_total : float
        The pairwise interaction energy of the ith particles with all other particles in the system
    """
    
    i_np = np.array(coordinates_np[i_particle])
    
    # Calculate distance
    distances = calculate_distance_np(i_np, coordinates_np, box_length)
    
    # Cutoff
    distances_cutoff = distances[distances < cutoff]
    
    # Non zero
    distance_nz = distances_cutoff[distances_cutoff != 0] 
       
    return calculate_lj_np(distance_nz)



In [34]:
coordinates = [[0, 0, 0], [0, 0, 2**(1/6)], [0, 0, 2*(2**(1/6))]]

coordinates_np = np.array(coordinates)

assert calculate_pair_energy_np(coordinates_np, 1, 10, 3) == -2

assert calculate_pair_energy_np(coordinates_np, 0, 10, 3) == calculate_pair_energy_np(coordinates_np, 2, 10, 3)

assert calculate_pair_energy_np(coordinates_np, 0, 10, 2) == -1

In [35]:
coordinates, box_length = read_xyz('../../lj_sample_configurations/lj_sample_config_periodic1.txt')

coordinates_np = np.array(coordinates)

start = time.time()
# Energy calculations
total_energy = calculate_total_energy_np(coordinates_np, box_length, cutoff)
print(total_energy)
total_correction = calculate_tail_correction(num_particles, box_length, cutoff)
print(total_correction)
total_energy += total_correction


for step in range(num_steps):
    # 1. Randomly pick one of the particles.
    random_particle = random.randrange(num_particles)
    
    # 2. Calculate the interaction energy of the selected particle with the system.
    current_energy = calculate_pair_energy_np(coordinates_np, random_particle, box_length, cutoff)
    
    # 3. Generate a random x, y, z displacement.
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)
    
    # 4. Modify the coordinate of Nth particle by generated displacements.
    coordinates_np[random_particle][0] += x_rand
    coordinates_np[random_particle][1] += y_rand
    coordinates_np[random_particle][2] += z_rand
    
    # 5. Calculate the interaction energy of the moved particle with the system and store this value.
    proposed_energy = calculate_pair_energy_np(coordinates_np, random_particle, box_length, cutoff)
    delta_energy = proposed_energy - current_energy
    
    # 6. Calculate if we accept the move based on energy difference.
    accept = accept_or_reject(delta_energy, beta)
    
    # 7. If accepted, move the particle.
    if accept:
        total_energy += delta_energy
    else:
        #Move not accepted, roll back coordinates
        coordinates_np[random_particle][0] -= x_rand
        coordinates_np[random_particle][1] -= y_rand
        coordinates_np[random_particle][2] -= z_rand
    
    # 8. Print the energy if step is a multiple of freq.
    if step % freq == 0:
        print(step, total_energy/num_particles)
        
end = time.time()
elapsed_time = end - start
print(elapsed_time)

-4351.540194543873
-6.057400016606341e-08
0 -5.438215781031509
1000 -5.451092581191813
2000 -5.450810022133854
3000 -5.4600306674503365
4000 -5.413401083447907
5.361205101013184


# Time Comparison
The np is performed over 15 times faster than the standard library simulation. In my local machine the standard MC Simulation ran in 84.76475787162781 secs vs. the MC Simulation using NumPy in 5.708783149719238 secs. 